In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

In [3]:
first_order = np.array([
        [1],
        [1]
])
hessian = np.array([
    [1, 0.15],
    [0.15, 1]
])

In [4]:
def quadratic(w):
    constant = 0.6
    return float(0.6 + first_order.T @ w + 0.5 * w.T @ hessian @ w)

In [5]:
def gradient(w):
    return first_order + hessian @ w

In [8]:
def vectorize(x, y):
    return np.array([x, y]).reshape(-1, 1)

In [9]:
def fs():
    return FloatSlider(min=-0.9, max=0.9, value=0.5, continuous_update=False)

In [10]:
from scipy import optimize
import time

In [14]:
@interact(b1=fs(), b2=fs(), h1=0.78, h2=-0.2, h3=-0.3, h4=fs())
def plot(b1, b2, h1, h2, h3, h4):
    
    first_order = np.array([
        [b1],
        [b2]
    ])
    hessian = np.array([
        [h1, h2],
        [h3, h4]
    ])
    
    def quadratic(w):
        return float(first_order.T @ w + 0.5 * w.T @ hessian @ w)
    
    def gradient(w):
        return first_order + hessian @ w
    
    def get_search_y(xs, starting_point, direction):
        return direction[1] / direction[0] * (xs - starting_point[0]) + starting_point[1]
        
    starting_point = np.array([[3], [3]])
    
    for i in range(10):
        
        direction = gradient(starting_point)

        def func(x):
            return np.abs(float(gradient(vectorize(x, get_search_y(x, starting_point, direction))).T @ direction))

        res = optimize.minimize(func, x0=0)

        grad_at_argmin = gradient(vectorize(res.x, get_search_y(res.x, starting_point, direction)))
        error_at_argmin = quadratic(vectorize(res.x, get_search_y(res.x, starting_point, direction)))

        plt.figure(figsize=(7, 7))

        plt.plot(np.linspace(-15, 15, 2), get_search_y(np.linspace(-15, 15, 2), starting_point, direction), '--', color='black', label='Search direction')
        plt.scatter(res.x, get_search_y(res.x, starting_point, direction), c='red', label='Point of minimum error')

        y_bounds = get_search_y(np.linspace(-3, 3, 2), starting_point, direction)

        xs = np.linspace(-15, 15, 200)
        ys = np.linspace(-15, 15, 200)
        xxs, yys = np.meshgrid(xs, ys)
        xxs, yys = xxs.flatten(), yys.flatten()

        ws = np.hstack([xxs.reshape(-1, 1), yys.reshape(-1, 1)])

        errors = []
        for w in ws:
            errors.append(float(quadratic(w.reshape(-1, 1))))

        if error_at_argmin < np.min(errors):
            error_at_argmin = np.min(errors) + 1
        if error_at_argmin > np.max(errors):
            error_at_argmin = np.max(errors) - 1

        plt.contourf(
            xxs.reshape(200, 200), 
            yys.reshape(200, 200), 
            np.array(errors).reshape(200, 200), 
            levels=list(np.concatenate([
                np.linspace(np.min(errors), error_at_argmin, 5),
                np.linspace(error_at_argmin+1e-7, np.max(errors), 10)
            ])),
            cmap='inferno',
            alpha=0.5
        )

        if not (grad_at_argmin[0] == 0 or grad_at_argmin[1] == 0):
            norm = np.linalg.norm(grad_at_argmin)
            plt.arrow(float(res.x), float(get_search_y(res.x, starting_point, direction)), -2*float(grad_at_argmin[0])/norm, -2*float(grad_at_argmin[1])/norm, length_includes_head=False, head_width=0.5, color='red')
        plt.xlim(-15, 15); plt.ylim(-15, 15)
        plt.axis('off')
        plt.legend()
        plt.show()

        try:
            angle_btw_search_direction_and_new_grad = np.rad2deg(np.arccos(
                float(direction.T @ grad_at_argmin) / (np.linalg.norm(direction) * np.linalg.norm(grad_at_argmin))
            ))
            print(f'Angle between search direction and new gradient: {angle_btw_search_direction_and_new_grad}')
        except:
            print('Error')

        try:
            np.linalg.cholesky(hessian)
            print('Hessian status: Hessian is positive definite.')
        except:
            print('Hessian status: Hessian is NOT positive definite.')
            
        time.sleep(3)
        starting_point = vectorize(res.x, get_search_y(res.x, starting_point, direction))

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='b1', max=0.9, min=-0.9), Fl…